In [6]:
import numpy as np
import cv2

In [7]:
# Apply Fourier Transform
def apply_fourier_transform(image):
    # Compute the 2D Fourier Transform
    f_transform = np.fft.fft2(image)
    
    # Shift the zero frequency component to the center
    f_transform_shifted = np.fft.fftshift(f_transform)
    
    # Compute the magnitude spectrum (log scale for better visualization)
    magnitude_spectrum = 20 * np.log(np.abs(f_transform_shifted))
    
    return magnitude_spectrum

# Step 3: Extract Features from Fourier Transform
def extract_features(magnitude_spectrum):
    # Flatten the magnitude spectrum for feature extraction
    flattened_spectrum = magnitude_spectrum.flatten()
    
    # Extract basic statistical features
    mean_val = np.mean(flattened_spectrum)
    std_dev = np.std(flattened_spectrum)
    skewness = np.mean((flattened_spectrum - mean_val) ** 3) / (std_dev ** 3)
    kurtosis = np.mean((flattened_spectrum - mean_val) ** 4) / (std_dev ** 4)
    
    # Return the features as a vector
    features = np.array([mean_val, std_dev, skewness, kurtosis])
    return features

In [8]:
# Step 1: Preprocess the Image
def preprocess_image(image_path):
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    image = cv2.resize(image, (128, 128))  # Resize for consistency
    return image

In [9]:
image = preprocess_image("C:/Users/sadik/OneDrive/Desktop/Liveness detection/dataset/Live/Train/60.jpg")
magnitude_spectrum = apply_fourier_transform(image) 
fourier_features = extract_features(magnitude_spectrum)
print(fourier_features)
print(fourier_features.shape)

[145.61475792  19.46870004   0.51327015   4.75888042]
(4,)


In [10]:
def extract_FT_features(image_path):
    image = preprocess_image(image_path)
    features=extract_features(image)
    return features

In [11]:
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix, accuracy_score
import os 
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score

In [12]:
folder_path = 'C:/Users/sadik/OneDrive/Desktop/Liveness detection/dataset'

In [13]:
def load_dataset(folder_path, label):
    feature_list = []
    labels=[]
    
    for image_file in os.listdir(folder_path):
        image_path = os.path.join(folder_path, image_file)
        features = extract_FT_features(image_path)
        feature_list.append(features)
        labels.append(label)
    
    return np.array(feature_list), np.array(labels)

In [14]:
live_train_path = 'C:/Users/sadik/OneDrive/Desktop/Liveness detection/dataset/Live/Train'
live_test_path = 'C:/Users/sadik/OneDrive/Desktop/Liveness detection/dataset/Live/Test'
spoof_train_path = 'C:/Users/sadik/OneDrive/Desktop/Liveness detection/dataset/Spoof/Train'
spoof_test_path = 'C:/Users/sadik/OneDrive/Desktop/Liveness detection/dataset/Spoof/Test'

In [15]:
#Extract feature for both live and spoof dataset 
X_live_train,y_live_train = load_dataset(live_train_path, label=1)
X_spoof_train, y_spoof_train = load_dataset(spoof_train_path,label=0)
X_live_test, y_live_test = load_dataset(live_test_path,label=1)
X_spoof_test, y_spoof_test = load_dataset(spoof_test_path,label=0)

In [16]:
# Combine the live and spoof training data
X_train = np.vstack((X_live_train, X_spoof_train))
y_train = np.concatenate((y_live_train, y_spoof_train), axis=0)

# Combine the live and spoof testing data
X_test = np.vstack((X_live_test, X_spoof_test))
y_test = np.concatenate((y_live_test, y_spoof_test), axis=0)

In [17]:
#Scale the features using StandardScaler
ss=StandardScaler()
X_train_scaled = ss.fit_transform(X_train)
X_test_scaled=ss.fit_transform(X_test)


In [18]:
# Check shapes to confirm consistency
print("Combined Train Shape:", X_train.shape, y_train.shape)

Combined Train Shape: (480, 4) (480,)


In [19]:
svc = SVC(kernel='rbf', C=1.0, random_state=42)  
svc.fit(X_train_scaled, y_train)



SVC(random_state=42)

In [20]:
#Forward feature selection=The Forward feature selection technique works in a way wherein at first a single feature is selected from the dataset and later all the features are added to the feature selection instance and later this instance object can be used to evaluate the model parameters.

import joblib
import sys
from mlxtend.feature_selection import SequentialFeatureSelector as sfs
from collections import Counter

class_counts = Counter(y_train)
sys.modules['sklearn.externals.joblib']=joblib
n_splits = min(class_counts.values())
forward_fs_best=sfs(estimator=svc,k_features='best',forward=True,verbose=1,scoring='accuracy',cv=2)


sfs_forward_best=forward_fs_best.fit(X_train_scaled,y_train)


c:\Users\sadik\anaconda3\envs\liveness\Lib\site-packages\sklearn\model_selection\_validation.py:73: FutureWarning: `fit_params` is deprecated and will be removed in version 1.6. Pass parameters via `params` instead.
  warnings.warn(
c:\Users\sadik\anaconda3\envs\liveness\Lib\site-packages\sklearn\model_selection\_validation.py:73: FutureWarning: `fit_params` is deprecated and will be removed in version 1.6. Pass parameters via `params` instead.
  warnings.warn(
c:\Users\sadik\anaconda3\envs\liveness\Lib\site-packages\sklearn\model_selection\_validation.py:73: FutureWarning: `fit_params` is deprecated and will be removed in version 1.6. Pass parameters via `params` instead.
  warnings.warn(
c:\Users\sadik\anaconda3\envs\liveness\Lib\site-packages\sklearn\model_selection\_validation.py:73: FutureWarning: `fit_params` is deprecated and will be removed in version 1.6. Pass parameters via `params` instead.
  warnings.warn(
Features: 1/4c:\Users\sadik\anaconda3\envs\liveness\Lib\site-package

In [21]:
print('R-Squared value:', sfs_forward_best.k_score_)

R-Squared value: 0.5979166666666667


In [22]:
# Predict on the test data
y_pred = svc.predict(X_test_scaled)

In [23]:
# Determine the unique labels in y_test
unique_labels = np.unique(y_test)

# Generate the confusion matrix with the appropriate labels
conf_matrix = confusion_matrix(y_test, y_pred, labels=unique_labels)

# Handle cases where unique_labels may not contain both 0 and 1
TP = conf_matrix[0][0] if 1 in unique_labels else 0
FN = conf_matrix[0][1] if 1 in unique_labels and len(unique_labels) > 1 else 0
FP = conf_matrix[1][0] if 0 in unique_labels and len(unique_labels) > 1 else 0
TN = conf_matrix[1][1] if 0 in unique_labels else 0

# Print the results
print("True Positive:", TP)
print("False Negative:", FN)
print("False Positive:", FP)
print("True Negative:", TN)
print("Accuracy of the SVM:", accuracy_score(y_test, y_pred))
print("Precision of the SVM:", precision_score(y_test, y_pred, pos_label=1) if 1 in unique_labels else 'N/A')
print("Recall of the SVM:", recall_score(y_test, y_pred, pos_label=1) if 1 in unique_labels else 'N/A')


True Positive: 47
False Negative: 13
False Positive: 26
True Negative: 34
Accuracy of the SVM: 0.675
Precision of the SVM: 0.723404255319149
Recall of the SVM: 0.5666666666666667
